In [38]:
#pytube is a python library to download youtube videos
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00


In [39]:
from pytube import YouTube

In [32]:
VIDEO_URL = 'https://www.youtube.com/watch?v=h-JVjs9AAmQ'

In [40]:
yt=YouTube(VIDEO_URL)  #downloading video

In [46]:
yt.streams \
  .filter(only_audio=True,file_extension='mp4')\
  .first()\
  .download(filename='ytaudio.mp4')

'/content/ytaudio.mp4'

to convert into text and then summarize we need wav file

In [57]:
!ffmpeg -i ytaudio.mp4 -vn -acodec pcm_s16le -ar 44100 -ac 2 ytaudio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

we downloaded audio of the youtube video
next is ASR AUTOMATIC SPEECH RECOGNITION OR SPEECH TO TEXT

In [48]:
!pip install huggingsound -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.12.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.12.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.12.1 whic

In [51]:
from huggingsound import SpeechRecognitionModel

In [49]:
import torch
device="cuda" if torch.cuda.is_available() else "cpu"

In [52]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)

INFO:huggingsound.speech_recognition.model:Loading model...


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

AUDIO CHUNKING IS SPLITTING THE AUDIO IN SMALLER AUDIOS TO CONVERT IT

In [53]:
import librosa

In [54]:
input_file='/content/ytaudio.wav'

In [58]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

44100


SILERO VAD IS USED IF WANT TO EXTRACT THHE AUDIO AND SPLIT IT INTO PERFECT PARTS WITHOUT SENTENCES BEING CUT

In [59]:
#to save the new files
import soundfile as sf

In [60]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav',speech,16000)

CHUNK TRANSCRIPTION

In [61]:
audio_path=[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav')

In [62]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav',
 '/content/7.wav',
 '/content/8.wav',
 '/content/9.wav',
 '/content/10.wav',
 '/content/11.wav',
 '/content/12.wav',
 '/content/13.wav',
 '/content/14.wav',
 '/content/15.wav',
 '/content/16.wav',
 '/content/17.wav',
 '/content/18.wav',
 '/content/19.wav',
 '/content/20.wav',
 '/content/21.wav',
 '/content/22.wav',
 '/content/23.wav']

In [63]:
transcriptions=model.transcribe(audio_path)

100%|██████████| 24/24 [16:09<00:00, 40.39s/it]


In [64]:
transcriptions

[{'transcription': 'arso fartasinutataazat fon te coas aastu asuatopta baege of supfostaution',
  'start_timestamps': [9300,
   9360,
   9580,
   9820,
   10540,
   10640,
   10860,
   10900,
   11460,
   11800,
   12520,
   12780,
   12840,
   13920,
   14460,
   14820,
   16060,
   16360,
   17060,
   17460,
   17860,
   18120,
   18260,
   18440,
   18720,
   18780,
   18900,
   18920,
   19040,
   19160,
   19200,
   19400,
   20440,
   20720,
   20860,
   22300,
   22600,
   22800,
   23180,
   23500,
   23940,
   24260,
   24500,
   24680,
   24980,
   25460,
   25740,
   25880,
   26200,
   26320,
   26440,
   26540,
   26740,
   26940,
   27140,
   27220,
   27300,
   27440,
   27480,
   27600,
   27680,
   27840,
   27920,
   28200,
   28440,
   28640,
   28800,
   29080,
   29760,
   29860,
   29880,
   29920,
   29940],
  'end_timestamps': [9320,
   9380,
   9600,
   9840,
   10560,
   10660,
   10880,
   10920,
   11480,
   11820,
   12540,
   12800,
   12860,
   13940,
   

In [65]:
#start with empty string and then join all the audio transcriptions and append in string
full_transcript=' '

In [67]:
#we get many things after transcription butwe need only description
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [68]:
full_transcript

' arso fartasinutataazat fon te coas aastu asuatopta baege of supfostautionuosteradk sumpannt fccai surduswatchmorbrich b brugshater cablucan evension sueha fomte owowry lash wot asrved for bop catalomeniser the blaatebishxe asit is used blahuch piihacacikeniaf farence abaction lgasiontaaasudenasam of tecakaasssacsiosa slap ofpaaserc afpifersto holnof soop assurancesuch afis ofaatalars retsestast inis yasiuhdushingeageaaversu aceshir at ra the souther tesimenlatiaish y bwiitulyu fortrabusos fosuaasutepagfusasor tosofsot sutor abaltusuus summurpartatstaat sasraatana patish ashaesantushonmanapatusuuctionaseantionbro asu te son smtthat astesuaansasalkurusmaf wasatefala shratafumba ratalombasanfmataumbmts look lausharkutmashes firs lan atetalarved atat achucraaxatatiuthe uplottaco cur but the love te wole soof aorvasson ofsioe apacusuat astsful sasu attuct by such as af tom at sasutmaretusaveofont batabumutaspsmuspulsmuor atfuae crustle  tersters w arer obay the battle baer chor shaboy te 

TEXT SUMMARIZATION

In [69]:
from transformers import pipeline

In [70]:
summarization=pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [71]:
summarization(full_transcript)['summary_text']

TypeError: list indices must be integers or slices, not str

TEXT CHUNKING

In [72]:
num_iters = int(len(full_transcript)/1000)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 1000
  end = (i + 1) * 1000
  #print("input text \n" + full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
  out = out[0]
  out = out['summary_text']
 # print("Summarized text\n"+out)
  summarized_text.append(out)

In [73]:
summarized_text

[' Arso fartasinutataazat fon te coas aastu as',
 ' aution mutrustue ba large tearf padias bibeshson babas']